```
This notebook sets up and runs a set of benchmarks to compare
different numerical discretizations of the SWEs

Copyright (C) 2016  SINTEF ICT

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

## Testing Playground for Open Boundary Conditions using Flow Relaxation Scheme

Flow Relaxation scheme is used in the benchmark cases in the MET report by Roed, and is also described in the compendia for GEF4510.
It is designed to simulate an open boundary, where energy is allowed to leave the domain of interest.
By altering the exterior solution, it also allows energy to enter the system as well.

The scheme is based on an extended computational region (similar to numerical sponge). A prediction solution, $\phi^*$ is computed as normally by the chosen numerical method, and the solution $phi^{n+1}$is adjusted by a interpolation factor towards the exterior solution, $\phi^E$, defined on the outermost ghost cells. This is done by

$$ \phi^{n+1} = (1-\alpha) \phi^* + \alpha\phi^E.$$

The factor $\alpha$ is found by 
$$ \alpha = 1 - \tanh\left(\frac{1-j}{3}\right)$$

Why did we use 3 instead of 2 in the denumerator in tanh? With 10 sponge cells, the alphas looked better, I think.


**This notebook** is intended as a playground for validating the Flow Relaxation implementations, documenting artifacts, test mixing boundary conditions, and so on.

If an interesting artifact is found, it should be kept unchanged so that it can be easily discussed with others.


###  Setting up the system

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec

import os
import pyopencl
import datetime
import sys

# requires netcdf4-python (netcdf4-python.googlecode.com)
from netCDF4 import Dataset as NetCDFFile

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Finally, import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, RecursiveCDKLM16, DataOutput, SimWriter, PlotHelper, Common
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

# Applying boundary conditions without simulation CDKLM

This example is ment to illustrate the numerical sponge boundary condition without doing simulation of it.
The inital condition is chosen as a frame in the outermost ghost cells of the computational domain, and the boundary condition will extrapolate  (interpolate) these outwards across the available ghost cells.

Here, we use 10 ghost cells in all directions.

In [ ]:
if 'sim' in globals():
    print "Found sim, so we will clean up!"
    sim.cleanUp()
    print " success :)"
else:
    print "Did not find sim in globals, and that's fine as well!"

reload(Common)
reload(CDKLM16)
    
#Coriolis well balanced reconstruction scheme  - NUMERICAL SPONGE TEST
nx = 10
ny = 20

dx = 200.0
dy = 200.0

dt = 0.95/5.0
g = 9.81

f = 0.00
r = 0.0
wind = Common.WindStressParams(type=99)

# Flow Relaxation Scheme sponge
ghosts = [10, 10, 10, 10]
boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=ghosts)
validDomain = [None, None, 0, 0]
    

dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

print ("dataShape from notebook: ", dataShape)

waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')

addCentralBump(h0, nx, ny, dx, dy, ghosts)
for x in range(nx + ghosts[1] + ghosts[3]):
    for y in range(ny + ghosts[0] + ghosts[2]):
        if ((x > 0 and x < ghosts[3]) or \
            (x > nx + ghosts[3]-1 and x < nx + ghosts[3] + ghosts[1] - 1)) and \
            (y > 0 and y < ny + ghosts[2] + ghosts[0] - 1):
            h0[y,x] += 1
        if ((y > 0 and y < ghosts[2]) or \
            (y > ny + ghosts[2]-1 and y < ny + ghosts[2] + ghosts[0] -1)) and \
           (x >= ghosts[3] and x <= nx + ghosts[3]-1):
            h0[y,x] += 1

fig = plt.figure(figsize=(3,3))
#plt.imshow(h0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], interpolation="None")
plt.imshow(h0, interpolation="None")
plt.colorbar()
plt.title("Initial contidions before applying FRS")

#Initialize simulator
reload(CDKLM16)
reload(KP07)
reload(Common)
#sim = KP07.KP07(cl_ctx, \
sim = CDKLM16.CDKLM16(cl_ctx, \
                h0, u0, v0, \
                Bi, \
                nx, ny, \
                dx, dy, dt, \
                g, f, r, \
                wind_stress=wind, \
                boundary_conditions=boundaryConditions)

t = sim.step(0.0)
h1, u1, v1 = sim.download()
fig = plt.figure(figsize=(3,3))
plt.imshow(h1, interpolation="None")
plt.colorbar()
plt.title("Initial conditions after applying FRS")

A gradually decreasing ellipse around the computational domain is correct behaviour

### Mixing boundary conditions
(without simulation)

Use the same initial conditions as above, but with a mix with other boundary conditions.

In [ ]:
if 'sim' in globals():
    sim.cleanUp()

reload(Common)
reload(CDKLM16)
    
#Coriolis well balanced reconstruction scheme  - NUMERICAL SPONGE TEST
nx = 10
ny = 20

dx = 200.0
dy = 200.0

dt = 0.95/5.0
g = 9.81

f = 0.00
r = 0.0
wind = Common.WindStressParams(type=99)


#ghosts = np.array([2,2,2,2]) # north, east, south, west
fig = plt.figure()
fig.title = "Mixing boundary conditions"
create_some_context
for i in range(7):
    if i == 0:
        msg = "Sponge in north"
        ghosts = [10, 2, 2, 2] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(3,2,1,2, spongeCells=ghosts)
    elif i == 1:
        msg = "Sponge in east"
        ghosts = [2, 10, 2, 2] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(2,3,2,1, spongeCells=ghosts)
    elif i == 2:
        msg = "Sponge in south"
        ghosts = [2, 2, 10, 2] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(1,2,3,2, spongeCells=ghosts)
    elif i == 3:
        msg = "Sponge in west"
        ghosts = [2, 2, 2, 10] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(2,1,2,3, spongeCells=ghosts)
    elif i == 4:
        msg = "Sponge in west and north"
        ghosts = [10, 2, 2, 10] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(3,1,1,3, spongeCells=ghosts)
    elif i == 5:
        msg = "Sponge in east and south"
        ghosts = [2, 10, 10, 2] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(1,3,3,1, spongeCells=ghosts)
    elif i == 6:
        msg = "Sponge in east and south and west"
        ghosts = [2, 10, 10, 10] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(1,3,3,3, spongeCells=ghosts)
    
    print "---------------\n" + msg
    validDomain = [None, None, 0, 0] # Plot all cells (including ghost cells)
    

    dataShape = (ny + ghosts[0]+ghosts[2], 
                 nx + ghosts[1]+ghosts[3])


    waterHeight = 60
    h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
    u0 = np.zeros(dataShape, dtype=np.float32, order='C');
    v0 = np.zeros(dataShape, dtype=np.float32, order='C');

    # Bathymetry:
    Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')

    addCentralBump(h0, nx, ny, dx, dy, ghosts)
    for x in range(nx + ghosts[1] + ghosts[3]):
        for y in range(ny + ghosts[0] + ghosts[2]):
            if ((x > 0 and x < ghosts[3]) or \
                (x > nx + ghosts[3]-1 and x < nx + ghosts[3] + ghosts[1] - 1)) and \
                (y > 0 and y < ny + ghosts[2] + ghosts[0] - 1):
                h0[y,x] += 1
            if ((y > 0 and y < ghosts[2]) or \
                (y > ny + ghosts[2]-1 and y < ny + ghosts[2] + ghosts[0] -1)) and \
               (x >= ghosts[3] and x <= nx + ghosts[3]-1):
                h0[y,x] += 1
    #print "From notebook, shape of h: ", h0.shape


    plt.subplot(7,2,i*2+1)
    #plt.imshow(h0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], interpolation="None")
    plt.imshow(h0, interpolation="None")
    plt.colorbar()
    plt.title(msg)

    #Initialize simulator
    reload(CDKLM16)
    reload(KP07)
    reload(Common)
    #sim = KP07.KP07(cl_ctx, \
    sim = CDKLM16.CDKLM16(cl_ctx, \
                    h0, u0, v0, \
                    Bi, \
                    nx, ny, \
                    dx, dy, dt, \
                    g, f, r, \
                    wind_stress=wind, \
                    boundary_conditions=boundaryConditions)

    t = sim.step(0.0)
    h1, u1, v1 = sim.download()
    sim.cleanUp()create_some_context
    plt.subplot(7,2,i*2+2)
    plt.imshow(h1, interpolation="None")
    plt.colorbar()
    plt.title(msg)

Mixing seems to work fine. Note that when ever possible (when opposing boundaries are not open (numerical sponge)) we use periodic boundary conditions. The opposing frame is then mapped on the ghost cells. 

## Simulation with numerical sponge

Here, we use a small domain with numerical sponge on all boundaries. The simulation is run for a long time, so that we expect the result lake-at-rest.

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
    
#Coriolis well balanced reconstruction scheme
nx = 30
ny = 60

dx = 200.0
dy = 200.0

dt = 0.95/5.0
g = 9.81

f = 0.00
r = 0.0
wind = Common.WindStressParams(type=99)

# Numerical Sponge
ghosts = [10, 10, 10, 10] # north, east, south, west
boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=ghosts)
validDomain = [None, None, 0, 0]
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
addLowerLeftBump(h0, nx, ny, dx, dy, ghosts)

u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')



#Initialize simulator
reload(CDKLM16)
reload(Common)
sim = CDKLM16.CDKLM16(cl_ctx, \
                h0, u0, v0, Bi,nx, ny, dx, dy, dt, g, f, r, \
                wind_stress=wind, boundary_conditions=boundaryConditions, \
                write_netcdf=False)

#Calculate radius from center of bump for plotting
x_center = dx*(nx+ghosts[0]+ghosts[2])/2.0
y_center = dy*(ny+ghosts[1]+ghosts[3])/2.0
y_coords, x_coords = np.mgrid[0:(ny+ghosts[0]+ghosts[2])*dy:dy, 0:(nx+ghosts[1]+ghosts[3])*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]], 
                                v0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]])

T = 100

def animate(i):
    if (i>0):
        #t = sim.step(10.0)
        t = sim.step(5.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()   
    brighten = 10 # Increase the values in the animation
    
    plotter.plot(brighten*(h1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]] - waterHeight), 
                 brighten*u1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]], 
                 brighten*v1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\t(Min, max) h: " + str((np.min(h1),np.max(h1))) + \
        "\tMax (u, v): " + str((np.max(u1), np.max(v1)))
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
#h2, u2, v2 = sim.download()
print "Initial water volume:      ", sum(sum(h0))
print "Initial bump volume:       ", sum(sum(h0 - waterHeight))
print "steady-state water volume: ", sum(sum(h2))
print "Water loss:                ", sum(sum(h0)) - sum(sum(h0 - waterHeight)) - sum(sum(h2))
sim.cleanUp()

**Results:** 

Case:
* netcdf: netcdf_2017_10_19/CDKLM16_2017_10_19-12_14_31.nc
* git commit: fe9427aa3df6014e55023839acb1d9936c439d29

Results:
* It can be seen on the animations for velocities ($u$, and $v$) that a small reflectation is generated. 
* Some alternating vecolicities lingeres on from the position of the initial bump. This should be irrelevent to the boundary conditions, but should still be investigated...
* There is a minor loss of water from steady state. This can be physically expected, as the outgoing wave might leave a wake behind it self, which will not be filled again.


Similar small test as above (relies on parameters from above), but with bump in the middle of the domain. This time also with Coriolis!

Make sure to run the example above before this one, in order to get the correct parameters.

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
    
    
print "Using (ny, nx): ", (ny, nx)
print "with ghosts: ", ghosts
#Coriolis well balanced reconstruction scheme
waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
addCentralBump(h0, nx, ny, dx, dy, ghosts)

f = 0.01

#Initialize simulator
reload(CDKLM16)
reload(Common)
sim = CDKLM16.CDKLM16(cl_ctx, \
                h0, u0, v0, Bi,nx, ny, dx, dy, dt, g, f, r, \
                wind_stress=wind, boundary_conditions=boundaryConditions, \
                write_netcdf=True)

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]], 
                                v0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]])

T = 250

def animate(i):
    if (i>0):
        #t = sim.step(10.0)
        t = sim.step(5.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()   
    brighten = 10 # Increase the values in the animation
    
    plotter.plot(brighten*(h1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]] - waterHeight), 
                 brighten*u1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]], 
                 brighten*v1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%30 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\t(Min, max) h: " + str((np.min(h1),np.max(h1))) + \
        "\tMax (u, v): " + str((np.max(u1), np.max(v1)))
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
#h2, u2, v2 = sim.download()
print "Initial water volume:      ", sum(sum(h0))
print "Initial bump volume:       ", sum(sum(h0 - waterHeight))
print "steady-state water volume: ", sum(sum(h2))
print "Water loss:                ", sum(sum(h0)) - sum(sum(h0 - waterHeight)) - sum(sum(h2))
sim.cleanUp()

**Results:** 

Case:
* netcdf: netcdf_2017_10_19/CDKLM16_2017_10_19-13_06_10.nc
* git commit: 35b5f14b963123d14f51f7671464326490026de1

Results:
* Waves leave the domain sufficiently well.
* The steady state solution is not completely round, but has small jumps in at 12, 3, 6 and 9 o'clock. Is there a bug in the CDKLM scheme?
* Oscillation velocities occures in the velocity plots. Not good.
* The steady state velocities are a bit wierd. E.g., for $u$ at approx $x=2000$, the solution for $y>0$ is large, small, large. Why is it large for $y=$ a few?

Further discussion:
* This issue has been investigated on two old versions of the code as well. In order to compensate the lack of numerical sponge, I tested two cases with $(nx, ny) = (300, 600)$, where the bump is added to the subdomain `addCentralBump(h0[267:333, 132:168], 30, 60, dx, dy, ghosts)`. The plot is made for the same area (in order to save resources and see the details).
* Version with 3 ghost cells (githash: d1b545575374260ae5e7ce8591291da018a33592): Similar to above.
* The old version with improper treatment of $U$ and $V$ (githash: badfd3fc2644fa51baa100c274e62e563bdacdb8): Terribel oscillations at first (because of the bug), but smooth and symmetric (nice looking) steady state solution.
* Now... Does KP07/CTCS behave similarly?

# KP07

### Mixing boundary conditions
(without simulation)

Use the same initial conditions as for CDKLM, where we use different degree of mixed boundary conditions and numerical sponge.

In [ ]:
if 'sim' in globals():
    sim.cleanUp()

reload(Common)
reload(KP07)
    
#Coriolis well balanced reconstruction scheme  - NUMERICAL SPONGE TEST
nx = 10
ny = 20

dx = 200.0
dy = 200.0

dt = 0.95/5.0
g = 9.81

f = 0.00
r = 0.0
wind = Common.WindStressParams(type=99)


#ghosts = np.array([2,2,2,2]) # north, east, south, west
fig = plt.figure()
fig.title = "Mixing boundary conditions"

for i in range(8):
    if i == 0:
        msg = "Sponge in all over"
        ghosts = [10, 10, 10, 10] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=ghosts)
    elif i == 1:
        msg = "Sponge in east"
        ghosts = [2, 10, 2, 2] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(2,3,2,1, spongeCells=ghosts)
    elif i == 2:
        msg = "Sponge in south"
        ghosts = [2, 2, 10, 2] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(1,2,3,2, spongeCells=ghosts)
    elif i == 3:
        msg = "Sponge in west"
        ghosts = [2, 2, 2, 10] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(2,1,2,3, spongeCells=ghosts)
    elif i == 4:
        msg = "Sponge in north"
        ghosts = [10, 2, 2, 2] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(3,2,1,2, spongeCells=ghosts)
    elif i == 5:
        msg = "Sponge in east and south"
        ghosts = [2, 10, 10, 2] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(1,3,3,1, spongeCells=ghosts)
    elif i == 6:
        msg = "Sponge in west and north"
        ghosts = [10, 2, 2, 10] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(3,1,1,3, spongeCells=ghosts)
    elif i == 7:
        msg = "Sponge in east and south and west"
        ghosts = [2, 10, 10, 10] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(1,3,3,3, spongeCells=ghosts)
    
    print "---------------\n" + msg
    
    validDomain = [None, None, 0, 0] # Plot all cells (including ghost cells)
    

    dataShape = (ny + ghosts[0]+ghosts[2], 
                 nx + ghosts[1]+ghosts[3])


    waterHeight = 60
    h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
    u0 = np.zeros(dataShape, dtype=np.float32, order='C');
    v0 = np.zeros(dataShape, dtype=np.float32, order='C');

    # Bathymetry:
    Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')

    addCentralBump(h0, nx, ny, dx, dy, ghosts)

    for x in range(nx + ghosts[1] + ghosts[3]):
        for y in range(ny + ghosts[0] + ghosts[2]):
            if not ((x == 0) or (x == nx + ghosts[1] + ghosts[3] - 1) or \
                    (y == 0) or (y == ny + ghosts[2] + ghosts[0] - 1)):
                h0[y,x] += 1 
                
    plt.subplot(8,2,i*2+1)
    #plt.imshow(h0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], interpolation="None")
    plt.imshow(h0, interpolation="None")
    plt.colorbar()
    plt.title(msg)

    #Initialize simulator
    reload(KP07)
    reload(Common)
    sim = KP07.KP07(cl_ctx, \
                    h0, Bi, u0, v0, \
                    nx, ny, \
                    dx, dy, dt, \
                    g, f, r, \
                    wind_stress=wind, \
                    boundary_conditions=boundaryConditions)

    t = sim.step(0.0)
    h1, u1, v1 = sim.download()
    sim.cleanUp()
    plt.subplot(8,2,i*2+2)
    plt.imshow(h1, interpolation="None")
    plt.colorbar()
    plt.title(msg)create_some_context

### Simulation with KP07 and FRS

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
    
#Coriolis well balanced reconstruction scheme
nx = 30
ny = 60

dx = 200.0
dy = 200.0

dt = 0.95/5.0
g = 9.81

f = 0.00
r = 0.0
wind = Common.WindStressParams(type=99)

# Numerical Sponge
ghosts = [10, 10, 10, 10] # north, east, south, west
boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=ghosts)
validDomain = [None, None, 0, 0]
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
addLowerLeftBump(h0, nx, ny, dx, dy, ghosts)

u0 = np.zeros(dataShape, dtype=np.float32, order='C');
v0 = np.zeros(dataShape, dtype=np.float32, order='C');

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')



#Initialize simulator
reload(KP07)
reload(Common)
sim = KP07.KP07(cl_ctx, \
                h0, Bi, u0, v0,nx, ny, dx, dy, dt, g, f, r, \
                wind_stress=wind, boundary_conditions=boundaryConditions, \
                write_netcdf=False)

#Calculate radius from center of bump for plotting
x_center = dx*(nx+ghosts[0]+ghosts[2])/2.0
y_center = dy*(ny+ghosts[1]+ghosts[3])/2.0
y_coords, x_coords = np.mgrid[0:(ny+ghosts[0]+ghosts[2])*dy:dy, 0:(nx+ghosts[1]+ghosts[3])*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]], 
                                v0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]])

T = 150

def animate(i):
    if (i>0):
        #t = sim.step(10.0)
        t = sim.step(5.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()   
    brighten = 5#create_some_context # Increase the values in the animation
    h1[:, ghosts[3]] = 61
    h1[:, nx + ghosts[3] - 1] = 61
    h1[ghosts[2], :] = 61
    h1[ny + ghosts[2] -1, :] = 61
    
    v1[:, ghosts[3]] = 61
    v1[:, nx + ghosts[3] - 1] = 61
    v1[ghosts[2], :] = 61
    v1[ny + ghosts[2] -1, :] = 61
    
    u1[:, ghosts[3]] = 61
    u1[:, nx + ghosts[3] - 1] = 61
    u1[ghosts[2], :] = 61
    u1[ny + ghosts[2] -1, :] = 61
    
    plotter.plot(brighten*(h1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]] - waterHeight), 
                 brighten*u1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]], 
                 brighten*v1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]]);
    fig.suptitle("KP07 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\t(Min, max) h: " + str((np.min(h1),np.max(h1))) + \
        "\tMax (u, v): " + str((np.max(u1), np.max(v1)))
        fig.savefig(imgdir + "/{:010.0f}_kp07.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
h2, u2, v2 = sim.download()
print "Initial water volume:      ", sum(sum(h0))
print "Initial bump volume:       ", sum(sum(h0 - waterHeight))
print "steady-state water volume: ", sum(sum(h2))
print "Water loss:                ", sum(sum(h0)) - sum(sum(h0 - waterHeight)) - sum(sum(h2))
sim.cleanUp()

**Results:** 

Case:
* netcdf: netcdf_2017_10_19/KP07_2017_10_19-17_24_53.nc
* git commit: 911e1114faf254b2395b8084bcff87103c650980

Results:
* Might be small reflections from boundary. Acceptable.

## Similar test with Coriolis

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
    
    
print "Using (ny, nx): ", (ny, nx)
print "with ghosts: ", ghosts
#Coriolis well balanced reconstruction scheme
waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
addCentralBump(h0, nx, ny, dx, dy, ghosts)

f = 0.01

#Initialize simulator
reload(KP07)
reload(Common)
sim = KP07.KP07(cl_ctx, \
                h0, Bi, u0, v0, nx, ny, dx, dy, dt, g, f, r, \
                wind_stress=wind, boundary_conditions=boundaryConditions, \
                write_netcdf=True)

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                h0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]] - waterHeight, 
                                u0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]], 
                                v0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]])

T = 250

def animate(i):
    if (i>0):
        #t = sim.step(10.0)
        t = sim.step(5.0)
    else:
        t = 0.0
    h1, u1, v1 = sim.download()   
    brighten = 10 # Increase the values in the animation
    
    plotter.plot(brighten*(h1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]] - waterHeight), 
                 brighten*u1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]], 
                 brighten*v1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]]);
    fig.suptitle("CDKLM16 Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%30 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\t(Min, max) h: " + str((np.min(h1),np.max(h1))) + \
        "\tMax (u, v): " + str((np.max(u1), np.max(v1)))
        fig.savefig(imgdir + "/{:010.0f}_cdklm16.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
sim.cleanUp()

**Results:** 

Case:
* netcdf: netcdf_2017_10_19/KP07_2017_10_19-17_28_24.nc
* git commit: 911e1114faf254b2395b8084bcff87103c650980

Results:
* Looks good, but the bump is touching the boundary, which might disturb the steady state! Perhaps this is also a problem in the test with CDKLM?

# Mixed boundary conditions for CTCS

In [ ]:
if 'sim' in globals():
    sim.cleanUp()

reload(Common)
reload(CTCS)
    
#Centered in time, centered in space scheme  - NUMERICAL SPONGE TEST
nx = 10
ny = 20

dx = 200.0
dy = 200.0

dt = 0.95/5.0
g = 9.81

f = 0.00
r = 0.0
A = 1.0
wind = Common.WindStressParams(type=99)


#ghosts = np.array([2,2,2,2]) # north, east, south, west
fig = plt.figure()
fig.title = "Mixing boundary conditions"

for i in range(8):
    if i == 0:
        msg = "Sponge in all over"
        ghosts = [10, 10, 10, 10] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(4,4,4,4, spongeCells=ghosts)
    if i == 1:
        msg = "Sponge in north"
        ghosts = [10, 2, 2, 2] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(4,2,1,2, spongeCells=ghosts)
    elif i == 2:
        msg = "Sponge in east"
        ghosts = [2, 10, 2, 2] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(2,4,2,1, spongeCells=ghosts)
    elif i == 3:
        msg = "Sponge in south"
        ghosts = [2, 2, 10, 2] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(1,2,4,2, spongeCells=ghosts)
    elif i == 4:
        msg = "Sponge in west"
        ghosts = [2, 2, 2, 10] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(2,1,2,4, spongeCells=ghosts)
    elif i == 5:
        msg = "Sponge in west and north"
        ghosts = [10, 2, 2, 10] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(4,1,1,4, spongeCells=ghosts)
    elif i == 6:
        msg = "Sponge in east and south"
        ghosts = [2, 10, 10, 2] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(1,4,4,1, spongeCells=ghosts)
    elif i == 7:
        msg = "Sponge in east and south and west"
        ghosts = [2, 10, 10, 10] # north, east, south, west
        boundaryConditions = Common.BoundaryConditions(1,4,4,4, spongeCells=ghosts)
    
    print "---------------\n" + msg
    validDomain = [None, None, 0, 0] # Plot all cells (including ghost cells)
    

    dataShape = (ny + ghosts[0]+ghosts[2], 
                 nx + ghosts[1]+ghosts[3])


    waterHeight = 60
    h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
    eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
    u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
    v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');
    
    addCentralBump(eta0, nx, ny, dx, dy, ghosts)
    for x in range(nx + ghosts[1] + ghosts[3]):
        for y in range(ny + ghosts[0] + ghosts[2]):
            if (x == ghosts[3] or x == nx + ghosts[3]-1) and \
               (y > ghosts[2] and y < ny + ghosts[2]-1):
                eta0[y,x] += 1
            if (y == ghosts[2] or y == ny + ghosts[2]-1) and \
               (x >= ghosts[3] and x <= nx + ghosts[3]-1):
                eta0[y,x] += 1
    for x in range(nx+1 + ghosts[1] + ghosts[3]):
        for y in range(ny + ghosts[0] + ghosts[2]):
            if (x == ghosts[3] or x == nx+1 + ghosts[3]-1) and \
               (y > ghosts[2] and y < ny + ghosts[2]-1):
                u0[y,x] += 0.1*y*x
            if (y == ghosts[2] or y == ny + ghosts[2]-1) and \
               (x >= ghosts[3] and x <= nx+1 + ghosts[3]-1):
                u0[y,x] += 0.1*x*y
    for x in range(nx + ghosts[1] + ghosts[3]):
        for y in range(ny+1 + ghosts[0] + ghosts[2]):
            if (x == ghosts[3] or x == nx + ghosts[3]-1) and \
               (y > ghosts[2] and y < ny+1 + ghosts[2]-1):
                v0[y,x] -= 1
            if (y == ghosts[2] or y == ny+1 + ghosts[2]-1) and \
               (x >= ghosts[3] and x <= nx + ghosts[3]-1):
                v0[y,x] -= 1
    #print "From notebook, shape of h: ", h0.shape


    plt.subplot(8,2,i*2+1)
    #plt.imshow(h0[ghosts[2]:-ghosts[0], ghosts[3]:-ghosts[1]], interpolation="None")
    plt.imshow(u0, interpolation="None")
    plt.colorbar()
    plt.title(msg)

    #Initialize simulator
    reload(CTCS)
    sim = CTCS.CTCS(cl_ctx, \
                   h0, eta0, u0, v0, \
                   nx, ny, \
                   dx, dy, dt, \
                   g, f, r, A, \
                   wind_stress=wind, \
                   boundary_conditions=boundaryConditions)

    t = sim.step(0.0)
    eta1, u1, v1 = sim.download()
    sim.cleanUp()    
    plt.subplot(8,2,i*2+2)
    plt.imshow(u1, interpolation="None")
    plt.colorbar()
    plt.title(msg)

### Simulation with open boundary and CTCS

In [ ]:
if 'sim' in globals():
    sim.cleanUp()
    
#Coriolis well balanced reconstruction scheme
nx = 30
ny = 60

dx = 200.0
dy = 200.0

dt = 0.95/5.0
g = 9.81
A = 1.0

f = 0.00
r = 0.0
wind = Common.WindStressParams(type=99)

# Numerical Sponge
ghosts = [10, 10, 10, 10] # north, east, south, west
boundaryConditions = Common.BoundaryConditions(4,4,4,4, spongeCells=ghosts)
validDomain = [None, None, 0, 0]
    
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])

waterHeight = 60
h0 = np.ones(dataShape, dtype=np.float32, order='C') * waterHeight;
eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');
addLowerLeftBump(eta0, nx, ny, dx, dy, ghosts)


#Initialize simulator
reload(CTCS)
reload(Common)
sim = CTCS.CTCS(cl_ctx, \
                   h0, eta0, u0, v0, \
                   nx, ny, dx, dy, dt, \
                   g, f, r, A, \
                   wind_stress=wind, \
                   boundary_conditions=boundaryConditions,
                   write_netcdf=True)

#Calculate radius from center of bump for plotting
x_center = dx*(nx+ghosts[0]+ghosts[2])/2.0
y_center = dy*(ny+ghosts[1]+ghosts[3])/2.0
y_coords, x_coords = np.mgrid[0:(ny+ghosts[0]+ghosts[2])*dy:dy, 0:(nx+ghosts[1]+ghosts[3])*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

fig = plt.figure()
plotter = PlotHelper.PlotHelper(fig, x_coords, y_coords, radius, 
                                eta0[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]] , 
                                u0[validDomain[2]:validDomain[0], validDomain[3]+1:validDomain[1]], 
                                v0[validDomain[2]+1:validDomain[0], validDomain[3]:validDomain[1]])

T = 100

def animate(i):
    if (i>0):
        #t = sim.step(10.0)
        t = sim.step(5.0)
    else:
        t = 0.0
    eta1, u1, v1 = sim.download()   
    brighten = 1 # Increase the values in the animation
    
    plotter.plot(brighten*(eta1[validDomain[2]:validDomain[0], validDomain[3]:validDomain[1]]), 
                 brighten*u1[validDomain[2]:validDomain[0], validDomain[3]+1:validDomain[1]], 
                 brighten*v1[validDomain[2]+1:validDomain[0], validDomain[3]:validDomain[1]]);
    fig.suptitle("CTCS Time = " + "{:04.0f}".format(t) + " s", fontsize=18)

    if (i%10 == 0):
        print "{:03.0f}".format(100*i / T) + " % => t=" + str(t) + "\t(Min, max) h: " + str((np.min(eta1),np.max(eta1))) + \
        "\tMax (u, v): " + str((np.max(u1), np.max(v1)))
        fig.savefig(imgdir + "/{:010.0f}_ctcs.png".format(t))
             
anim = animation.FuncAnimation(fig, animate, range(T), interval=100)
plt.close(anim._fig)
anim

In [ ]:
sim.cleanUp()

**Results:** 

Case:
* netcdf: netcdf_2017_10_23/CTCS_2017_10_23-18_54_34.nc
* git commit: b49391e7d3ca554aa04f6309556a1e719dde4f36

Results:
* Does not look good...
* eta looks okay
* U does not work to the west, and the east
* V does not work to the south and north

In [ ]:
x = np.linspace(1, 10, 10)
plt.plot(x, 1 - np.tanh((x-1.0)/2.7), '*')
for i in x:
    print (i, 1 - np.tanh((i-1)/2.0), 1 - np.tanh((i-1)/2.7), 1 - np.tanh((i-1)/3.))
